In [0]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2 as cv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


import time
import random
import matplotlib.pyplot as plt
from collections import deque

from PIL import Image
from scipy import ndimage
import copy
from PIL import Image as PILImage
import math

In [0]:
import cv2 as cv
from google.colab.patches import cv2_imshow
from PIL import Image
from scipy import ndimage
import copy
from PIL import Image as PILImage
import numpy as np
import math


class car(object):

  # x and y are center points of the car

    def __init__(
        self,
        x,
        y,
        angle,
        ):
        self.x = x
        self.y = y
        (self.length, self.width) = (int(20), int(10))
        self.angle = angle

    def move(
        self,
        velocity_x,
        velocity_y,
        rotation,
        ):
        self.x = self.x + velocity_x
        self.y = self.y + velocity_y
        self.angle = self.angle + rotation

        if self.angle > 360:
            self.angle = self.angle % 360
        elif self.angle < -360:
            self.angle = self.angle % -360


class city(object):

    def __init__(self, city_file):
        self.city_file = city_file
        self.city_img = cv.imread(self.city_file)
        (self.width, self.length, _) = self.city_img.shape

    def draw_car(
        self,
        x,
        y,
        width,
        height,
        angle,
        img,
        ):

        _angle = (180 - angle) * math.pi / 180.0
        b = math.cos(_angle) * 0.5
        a = math.sin(_angle) * 0.5
        pt0 = (int(x - a * height - b * width), int(y + b * height - a
               * width))
        pt1 = (int(x + a * height - b * width), int(y - b * height - a
               * width))
        pt2 = (int(2 * x - pt0[0]), int(2 * y - pt0[1]))
        pt3 = (int(2 * x - pt1[0]), int(2 * y - pt1[1]))
        pt4 = (int((pt0[0] + pt1[0]) / 2), int((pt0[1] + pt1[1]) / 2))
        pt5 = (int((pt0[0] + pt3[0]) / 2), int((pt0[1] + pt3[1]) / 2))
        pt6 = (int((pt1[0] + pt2[0]) / 2), int((pt1[1] + pt2[1]) / 2))

        line_color = (200, 200, 200)
        line_thickness = 5

    # print(pt0, pt1, pt2, pt3, pt4)

        cv.line(img, pt2, pt3, line_color, line_thickness)
        cv.line(img, pt3, pt5, line_color, line_thickness)
        cv.line(img, pt6, pt2, line_color, line_thickness)
        cv.line(img, pt5, pt4, line_color, line_thickness)
        cv.line(img, pt6, pt4, line_color, line_thickness)

    def get_current_loc_map(
        self,
        x,
        y,
        size,
        angle=0,
        state=False,
        ):

        newcity_img = copy.deepcopy(self.city_img)

        if x - size / 2 < 0 or y - size / 2 < 0 or x + size / 2 \
            > self.length - 1 or y + size / 2 > self.width - 1:
            return np.ones((size, size, 3))
        else:
            y = self.width - y

        if state == True:
            img_crop = self.draw_car(
                x,
                y,
                20,
                10,
                angle,
                newcity_img,
                )

        img_crop = newcity_img[int(y - size / 2):int(y) + int(size / 2), int(x
                               - size / 2):int(x) + int(size / 2)]
        img_state = np.average(img_crop, axis=2) / 255
        return (img_crop, img_state)


class env(object):

    def __init__(
        self,
        car,
        city,
        city_map,
        car_img,
        ):
        self.car = car
        self.city = city
        self.city_map = city_map
        self.car_img = car_img
        self.car_img = cv.resize(self.car_img, (self.car.length,
                                 self.car.width))
        self.size = 80

    # car_x and car_y are center points of the car

    def show_image(self):
        newcity = copy.deepcopy(self.city)
        car_rotated = ndimage.rotate(self.car_img, self.car.angle)
        (car_wid, car_len, _) = car_rotated.shape
        pos_x = self.car.x - car_len // 2
        pos_y = newcity.width - (self.car.y - car_wid // 2)

        if pos_x < 0:
            pos_x = 0
        elif pos_x > newcity.length:
            pos_x = newcity.length - car_len

        if pos_y > newcity.width:
            pos_y = newcity.width - car_wid
        elif pos_y < 0:
            pos_y = 0

        car_rotated = cv.addWeighted(newcity.city_img[pos_y:pos_y
                + car_wid, pos_x:pos_x + car_len], 0.5, car_rotated, 1,
                0)
        newcity.city_img[pos_y:pos_y + car_wid, pos_x:pos_x
                         + car_len] = car_rotated
        return newcity

    def step(self, action):
        self.reward = 0
        self.velocity_x = 0.5
        self.velocity_y = 0
        done = False

        angle = math.radians(action)
        self.velocity_x = self.velocity_x * math.cos(angle) \
            - self.velocity_y * math.sin(angle)
        self.velocity_y = self.velocity_y * math.cos(angle) \
            + self.velocity_x * math.sin(angle)
        self.car.move(self.velocity_x, self.velocity_y, action)
        xx = self.goal_x - self.car.x
        yy = self.goal_y - self.car.y

        distance = np.sqrt((self.car.x - self.goal_x) ** 2 + (self.car.y
                           - self.goal_y) ** 2)
    
        car_loc, _ = self.city_map.get_current_loc_map(self.car.x,
                              self.car.y, self.size)
        sand_quality = np.sum(car_loc)
        sand_quality = sand_quality / (self.size * self.size * 3 * 255)
    
        # moving on the sand
        # check coordinates carefully image cordinate y is inverse of car coordinate y
    
        sand_check = np.sum(self.city_map.city_img[int(self.city.width
                            - self.car.y), int(self.car.x)]) / (255 * 3)
    
        if sand_check > 0:  # **** Check whether coords are correct
            self.reward = self.reward - 5.0
        else:
    
               # moving on the road
    
            self.reward = self.reward - 1.5
    
        if self.car.x - int(self.car.length / 2) < 5 or self.car.y \
            - int(self.car.width / 2) < 5 or self.car.x \
            - int(self.car.length / 2) > self.city_map.length - 5 \
            or self.car.y - int(self.car.width / 2) > self.city_map.width \
            - 5:
            self.boundary_hit_count = self.boundary_hit_count + 1
            self.reward = self.reward - 5.0
    
        if distance < self.last_distance:
            self.reward = self.reward + 5
        else:
            self.reward = self.reward + 2
    
        if distance < 25:
            self.reward = self.reward + 100
    
            self.goal_hit_count += 1
    
            if swap == 1:
                print ('Hit the Goal 2: (' + str(goal_x) + ', ' \
                    + str(goal_y) + ')')
                traversal_log.write('Train episode: '
                                    + str(train_episode_num)
                                    + ' Eval episode: '
                                    + str(eval_episode_num)
                                    + ' : Hit the Goal 2: (' + str(goal_x)
                                    + ', ' + str(goal_y) + ')\n')
                self.goal_x = 575
                self.goal_y = 530
                self.swap = 0
                done = True
            else:
                print ('Hit the Goal 1: (' + str(goal_x) + ', ' \
                    + str(goal_y) + ')')
                traversal_log.write('Train episode: '
                                    + str(train_episode_num)
                                    + ' Eval episode: '
                                    + str(eval_episode_num)
                                    + ' : Hit the Goal 1: (' + str(goal_x)
                                    + ', ' + str(goal_y) + ')\n')
                self.goal_x = 610
                self.goal_y = 45
                self.swap = 1
                done = True
    
        self.last_distance = distance
        self.current_step += 1
    
        img_crop = self.city.get_current_loc_map(self.car.x, self.car.y,
                self.car.angle, state=True)
        self.last_action = action
        self.last_reward = self.reward
    
        return (img_crop, self.reward, done)

    def reset(self):

        longueur = self.city.length
        largeur = self.city.width

        self.on_road_count = 0
        self.off_road_count = 0
        self.boundary_hit_count = 0
        self.goal_hit_count = 0
        self.episode_total_reward = 0.0
        self.reward = 0
        self.last_reward = 0
        self.last_action = 0
        self.goal_x = 575
        self.goal_y = 530
        self.swap = 0
        self.last_distance = 0
        self.current_step = 0

        self.car.angle = 0.0
        self.car.x = np.random.randint(100, longueur - 100)
        self.car.y = np.random.randint(100, largeur - 100)


In [0]:
car_file = 'car.jpg'
city_file = 'citymap.png'
city_map_file = "MASK1.png"
car_img = cv.imread(car_file)
car1 = car(0,0,0)
city1 = city(city_file)
citymap1 = city(city_map_file)

In [0]:
# Step 1: Initialize the Experience Replay memory
# state[i] = [cropped image state] + [distance to target state]

class ReplayBuffer(object):

    def __init__(self, max_size=1e6):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, transition):
        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = transition
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(transition)

    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        (batch_states, batch_next_states, batch_actions, batch_rewards,
         batch_dones) = ([], [], [], [], [])
        for i in ind:
            (state, next_state, action, reward, done) = self.storage[i]
            batch_states.append(np.array(state, copy=False))
            batch_next_states.append(np.array(next_state, copy=False))
            batch_actions.append(np.array(action, copy=False))
            batch_rewards.append(np.array(reward, copy=False))
            batch_dones.append(np.array(done, copy=False))

        return (np.array(batch_states), np.array(batch_next_states),
                np.array(batch_actions),
                np.array(batch_rewards).reshape(-1, 1),
                np.array(batch_dones).reshape(-1, 1))


# Step 2: Define CNN network for the Actor model
# Same network is to be used for the Actor target

class Actor(nn.Module):

    def __init__(
        self,
        state_dim,
        action_dim,
        max_action,
        ):
        super(Actor, self).__init__()

    # input state image size will 40*40*1

        self.convblock1 = nn.Sequential(nn.Conv2d(in_channels=1,
                out_channels=8, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(8), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 38

        self.convblock2 = nn.Sequential(nn.Conv2d(in_channels=8,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 36

        self.convblock3 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # putput_size = 34

        self.pool1 = nn.MaxPool2d(2, 2)  # output_size = 17
        self.convblock4 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=8, kernel_size=(1, 1), padding=0,
                bias=False), nn.BatchNorm2d(8), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 17

        self.convblock5 = nn.Sequential(nn.Conv2d(in_channels=8,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 15

        self.convblock6 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=32, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(32), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 13

        self.convblock7 = nn.Sequential(nn.Conv2d(in_channels=32,
                out_channels=32, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(32), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 11

        self.GAP = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)))

    # GAP to return 32 and rest of state values to be added before passing to fully connected layer

        self.fc1 = nn.Linear(state_dim - 1 + 32, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, action_dim)

        self.max_action = max_action

    def forward(self, state):

        x = self.convblock1(state[0])
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.GAP(x)
        x = x.view(-1, 32)

    # concatenate with rest of the state elements

        x = torch.cat([x, state[1:]], 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.max_action * torch.tanh(self.fc3(x))
        return x


# Step 3: Define neural networks for the two Critic models and Critic targets
# Model will be same for all 4 critics

class Critic(nn.Module):

    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

    # Defining the first Critic CNN based network

        self.convblock1_1 = nn.Sequential(nn.Conv2d(in_channels=1,
                out_channels=8, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(8), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 38

        self.convblock1_2 = nn.Sequential(nn.Conv2d(in_channels=8,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 36

        self.convblock1_3 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # putput_size = 34

        self.pool1_1 = nn.MaxPool2d(2, 2)  # output_size = 17
        self.convblock1_4 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=8, kernel_size=(1, 1), padding=0,
                bias=False), nn.BatchNorm2d(8), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 17

        self.convblock1_5 = nn.Sequential(nn.Conv2d(in_channels=8,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 15

        self.convblock1_6 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=32, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(32), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 13

        self.convblock1_7 = nn.Sequential(nn.Conv2d(in_channels=32,
                out_channels=32, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(10), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 11

        self.GAP1_1 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)))

    # GAP to return 32 and rest of state values to be added before passing to fully connected layer

        self.fc1_1 = nn.Linear(state_dim - 1 + 32 + action_dim, 400)
        self.fc1_2 = nn.Linear(400, 300)
        self.fc1_3 = nn.Linear(300, 1)

    # Defining the second Critic neural network

        self.convblock2_1 = nn.Sequential(nn.Conv2d(in_channels=1,
                out_channels=8, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(8), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 38

        self.convblock2_2 = nn.Sequential(nn.Conv2d(in_channels=8,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 36

        self.convblock2_3 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # putput_size = 34

        self.pool2_1 = nn.MaxPool2d(2, 2)  # output_size = 17
        self.convblock2_4 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=8, kernel_size=(1, 1), padding=0,
                bias=False), nn.BatchNorm2d(8), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 17

        self.convblock2_5 = nn.Sequential(nn.Conv2d(in_channels=8,
                out_channels=16, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(16), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 15

        self.convblock2_6 = nn.Sequential(nn.Conv2d(in_channels=16,
                out_channels=32, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(32), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 13

        self.convblock2_7 = nn.Sequential(nn.Conv2d(in_channels=32,
                out_channels=32, kernel_size=(3, 3), padding=0,
                bias=False), nn.BatchNorm2d(10), nn.ReLU(),
                nn.Dropout(0.1))  # output_size = 11

        self.GAP2_1 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)))

    # we will have 32 values coming from GAP layer and (state_dim-1) other state values

        self.fc2_1 = nn.Linear(state_dim - 1 + 32 + action_dim, 400)
        self.fc2_2 = nn.Linear(400, 300)
        self.fc2_3 = nn.Linear(300, 1)

    def forward(self, state, action):

    # first state element is cropped image
    # Forward-Propagation of the first Critic Network

        x1 = self.convblock1_1(state[0])
        x1 = self.convblock1_2(x1)
        x1 = self.convblock1_3(x1)
        x1 = self.pool1_1(x1)
        x1 = self.convblock1_4(x1)
        x1 = self.convblock1_5(x1)
        x1 = self.convblock1_6(x1)
        x1 = self.convblock1_7(x1)
        x1 = self.GAP1_1(x1)
        x1 = x1.view(-1, 32)

    # concatenate with rest of the state elements

        x1 = torch.cat([x1, state[1:], action], 1)
        x1 = F.relu(self.fc1_1(x1))
        x1 = F.relu(self.fc1_2(x1))
        x1 = self.fc1_3(x1)

    # Forward-Propagation of the second Critic Network

        x2 = self.convblock2_1(state[0])
        x2 = self.convblock2_2(x2)
        x2 = self.convblock2_3(x2)
        x2 = self.pool2_1(x2)
        x2 = self.convblock2_4(x2)
        x2 = self.convblock2_5(x2)
        x2 = self.convblock2_6(x2)
        x2 = self.convblock2_7(x2)
        x2 = self.GAP2_1(x2)
        x2 = x2.view(-1, 32)

    # concatenate with rest of the state elements

        x2 = torch.cat([x2, state[1:], action], 1)
        x2 = F.relu(self.fc2_1(x2))
        x2 = F.relu(self.fc2_2(x2))
        x2 = self.fc2_3(x2)

        return (x1, x2)

    def Q1(self, state, action):

    # Forward-Propagation of the first Critic Network

        x1 = self.convblock1_1(state[0])
        x1 = self.convblock1_2(x1)
        x1 = self.convblock1_3(x1)
        x1 = self.pool1_1(x1)
        x1 = self.convblock1_4(x1)
        x1 = self.convblock1_5(x1)
        x1 = self.convblock1_6(x1)
        x1 = self.convblock1_7(x1)
        x1 = self.GAP1_1(x1)
        x1 = x1.view(-1, 32)

    # concatenate with rest of the state elements

        x1 = torch.cat([x1, state[1:], action], 1)
        x1 = F.relu(self.fc1_1(x1))
        x1 = F.relu(self.fc1_2(x1))
        x1 = self.fc1_3(x1)

        return x1


# Defining new class for step 4 to step 15
# complete training Process is in the given class

class TD3(object):

    def __init__(
        self,
        state_dim,
        action_dim,
        max_action,
        ):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim,
                                  max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = \
            torch.optim.Adam(self.critic.parameters())
        self.max_action = max_action

    def select_action(self, state):
        state = torch.Tensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(
        self,
        replay_buffer,
        iterations,
        batch_size=100,
        discount=0.99,
        tau=0.005,
        policy_noise=0.2,
        noise_clip=0.5,
        policy_freq=2,
        ):

        for it in range(iterations):

      # Step 4: We sample a batch of transitions (s, s’, a, r) from the memory

            (batch_states, batch_next_states, batch_actions,
             batch_rewards, batch_dones) = \
                replay_buffer.sample(batch_size)
            state = torch.Tensor(batch_states).to(device)
            next_state = torch.Tensor(batch_next_states).to(device)
            action = torch.Tensor(batch_actions).to(device)
            reward = torch.Tensor(batch_rewards).to(device)
            done = torch.Tensor(batch_dones).to(device)

      # Step 5: From the next state s’, the Actor target plays the next action a’

            next_action = self.actor_target(next_state)

      # Step 6: We add Gaussian noise to this next action a’ and we clamp it in a range of values supported by the environment

            noise = torch.Tensor(batch_actions).data.normal_(0,
                    policy_noise).to(device)
            noise = noise.clamp(-noise_clip, noise_clip)
            next_action = (next_action + noise).clamp(-self.max_action,
                    self.max_action)

      # Step 7: The two Critic targets take each the couple (s’, a’) as input and return two Q-values Qt1(s’,a’) and Qt2(s’,a’) as outputs

            (target_Q1, target_Q2) = self.critic_target(next_state,
                    next_action)

      # Step 8: We keep the minimum of these two Q-values: min(Qt1, Qt2)

            target_Q = torch.min(target_Q1, target_Q2)

      # Step 9: We get the final target of the two Critic models, which is: Qt = r + γ * min(Qt1, Qt2), where γ is the discount factor

            target_Q = reward + ((1 - done) * discount
                                 * target_Q).detach()

      # Step 10: The two Critic models take each the couple (s, a) as input and return two Q-values Q1(s,a) and Q2(s,a) as outputs

            (current_Q1, current_Q2) = self.critic(state, action)

      # Step 11: We compute the loss coming from the two Critic models: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)

            critic_loss = F.mse_loss(current_Q1, target_Q) \
                + F.mse_loss(current_Q2, target_Q)

      # Step 12: We backpropagate this Critic loss and update the parameters of the two Critic models with a SGD optimizer

            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

      # Step 13: Once every two iterations, we update our Actor model by performing gradient ascent on the output of the first Critic model

            if it % policy_freq == 0:
                actor_loss = -self.critic.Q1(state,
                        self.actor(state)).mean()
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                self.actor_optimizer.step()

        # Step 14: Still once every two iterations, we update the weights of the Actor target by polyak averaging

                for (param, target_param) in \
                    zip(self.actor.parameters(),
                        self.actor_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1
                            - tau) * target_param.data)

        # Step 15: Still once every two iterations, we update the weights of the Critic target by polyak averaging

                for (param, target_param) in \
                    zip(self.critic.parameters(),
                        self.critic_target.parameters()):
                    target_param.data.copy_(tau * param.data + (1
                            - tau) * target_param.data)

  # Making a save method to save a trained model

    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '%s/%s_actor.pth'
                   % (directory, filename))
        torch.save(self.critic.state_dict(), '%s/%s_critic.pth'
                   % (directory, filename))

  # Making a load method to load a pre-trained model

    def load(self, filename, directory):
        self.actor.load_state_dict(torch.load('%s/%s_actor.pth'
                                   % (directory, filename)))
        self.critic.load_state_dict(torch.load('%s/%s_critic.pth'
                                    % (directory, filename)))


**Function to evaluates the policy by calculating its average reward over 10 episodes**


In [0]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

**Set required parameters**

In [0]:
env = env(car1, city1, citymap1, car_img) # Instantiate the environment
seed = 0 # Random seed number
start_timesteps = 1e4 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = True # Boolean checker whether or not to save the pre-trained model
expl_noise = 0.1 # Exploration noise - STD value of exploration Gaussian noise
batch_size = 100 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.005 # Target network update rate
policy_noise = 0.2 # STD of Gaussian noise added to the actions for the exploration purposes
noise_clip = 0.5 # Maximum value of the Gaussian noise added to the actions (policy)
policy_freq = 2 # Number of iterations to wait before the policy network (Actor model) is updated

**Define file name for the two saved models: the Actor and Critic models**

In [7]:
file_name = "%s_%s" % ("TD3_car", str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Settings: TD3_car_0
---------------------------------------


**Create a folder to store the trained models**

In [0]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

**Set seeds and get necessary information on the states and actions**

In [0]:
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = 2
action_dim = 1
max_action = 10

In [10]:
env.reset()
action = 30
new_obs, reward, done = env.step(action)
print(new_obs)
print(done)

(array([[[127, 104,  61],
        [127, 104,  61],
        [127, 104,  61],
        ...,
        [240, 244, 248],
        [246, 252, 254],
        [198, 193, 190]],

       [[126, 103,  61],
        [126, 104,  61],
        [127, 104,  61],
        ...,
        [233, 236, 231],
        [235, 237, 233],
        [201, 203, 199]],

       [[128, 105,  61],
        [127, 103,  61],
        [126, 104,  61],
        ...,
        [148, 142, 141],
        [144, 134, 133],
        [126, 120, 119]],

       ...,

       [[119,  89,  57],
        [132, 104,  68],
        [134, 110,  70],
        ...,
        [147, 145, 144],
        [124, 124, 121],
        [123, 113, 113]],

       [[102,  72,  46],
        [119,  91,  55],
        [128, 101,  61],
        ...,
        [134, 129, 129],
        [127, 119, 122],
        [127, 115, 117]],

       [[ 90,  61,  38],
        [111,  81,  48],
        [129, 100,  60],
        ...,
        [160, 151, 154],
        [156, 146, 154],
        [136, 121, 123]